# 1917 영화 댓글 Preprocessing

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import math
import time
import re

## Naver 영화 페이지의 '1917'영화 댓글 Crawling

In [2]:
# Chromedriver로 chrome browser 실행
browser = webdriver.Chrome(executable_path=r'C:/chromedriver_win32/chromedriver.exe')

# Naver 영화 페이지의 '1917'영화 페이지 이동
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187321'
browser.get( url )

# 평점에 더보기 링크 클릭
view_more = browser.find_element_by_xpath( '//*[@id="content"]/div[1]/div[4]/div[5]/div[2]/div[1]/a' )
view_more.click()

browser.switch_to_frame( browser.find_element_by_id( 'pointAfterListIframe' ) )

C:\Users\user\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: use driver.switch_to.frame instead
  if sys.path[0] == '':


In [3]:
# 전체 댓글수에 따른 페이지 계산
html0 = browser.page_source
html1 = BeautifulSoup( html0, 'lxml' )
html2 = html1.find( 'div', { 'class': 'score_total' } ).find( 'strong' ).findChildren( 'em' )[ 0 ].getText()
total_comment = int( html2.replace(',', '') )
total_page = total_comment / 10

print( total_comment )
print( total_page )
print( math.trunc( total_page ) )

3104
310.4
310


In [4]:
movie_comments = []

for page in range( 0, math.trunc( total_page ) ): # 전체 페이지를 이동
    time.sleep( 1 ) #1초의 시간 delay
    html0 = browser.page_source #selenuim에서 현재 있는 browser의 페이지 소스 불러오기
    html1 = BeautifulSoup( html0, 'lxml') #bs4를 이용하여 html로 파싱하기
    html2 = html1.find( 'div', { 'class': 'ifr_area basic_ifr' } ) #댓글을 포함하고 있는 tag찾기
    review0 = html2.find( 'div', { 'class': 'score_result' } ).find_all( 'li' ) #각 댓글별로 list화
    
    for i in range( len( review0 ) ): #한페이지 내에서 모든 댓글을 수집하기 위한 반복문
        review = review0[ i ].find( 'div',{ 'class': 'score_reple' } ).find( 'span' ).text.strip() #댓글
        if review not in '관람객': # 불필요한 내용 삭제
            movie_comments.append( review + '\n' )
        
    #다음페이지로 넘어가기
    if page == 0:
        browser.find_elements_by_xpath('//*[@class = "paging"]/div/a')[10].click()
    else:
        browser.find_elements_by_xpath('//*[@class = "paging"]/div/a')[11].click()


In [5]:
print(movie_comments)

['충무로: 이거 어케하는거냐?\n', '촬영감독의 영혼까지 갈아넣은 마스터피스\n', '오스카 작품상 탔어도 할말 없었을것 같다.\n', '주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차대전에 참전하는 느낌.. 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참.....\n', '닥터스트레인지의 판단력이 좋았다.\n', '솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다. 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽.\n', '아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음. 감독의 참신성이 빛을 발함. 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이...\n', '스포일러가 포함된 감상평입니다. 감상평 보기\n', '진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...\n', '솔직히 나는 기생충보다 이영화가 더 재미있었다.\n', '막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요!\n', '원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음\n', '스포일러가 포함된 감상평입니다. 감상평 보기\n', '일단..극장에서 보는걸  추천드립니다.카메라감독 상 줘야 할판  엄청나게  잘 찍음\n', '솔직히 샘 멘데스 감독 좀 억울할듯기생충도 재밌게 봤지만, 이거 보고나서 좀 억울했겠다는 생각이 팍듬..\n', '동료를 잃고 목숨을 잃을 위기를 극복해나가며 달려 얻은 것은 전쟁의 무의미함과 허무함 뿐.\n', '전쟁이 끝나는 길은 마지막 한사람까지 죽는 것이다.\n', '기생충이 그냥 백인이 나온 헐리우드 영화였다면 절대 1917을 이기지 못했을것. 1917은 오히려 백인의 헐리우드 영화라 차별받음. 

In [6]:
# 파일 저장
with open('comment_1917.txt', 'w', encoding='utf-8') as f:
    for comment in movie_comments:
        f.write(comment + '\n')

In [2]:
# 파일에서 읽기
movie_comments = []
with open('comment_1917.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        str = line.rstrip('\n')
        if str != '':
            movie_comments.append(str)
            
print(movie_comments)

['충무로: 이거 어케하는거냐?', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.', '주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차대전에 참전하는 느낌.. 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참.....', '닥터스트레인지의 판단력이 좋았다.', '솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다. 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽.', '아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음. 감독의 참신성이 빛을 발함. 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이...', '스포일러가 포함된 감상평입니다. 감상평 보기', '진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...', '솔직히 나는 기생충보다 이영화가 더 재미있었다.', '막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요!', '원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음', '스포일러가 포함된 감상평입니다. 감상평 보기', '일단..극장에서 보는걸  추천드립니다.카메라감독 상 줘야 할판  엄청나게  잘 찍음', '솔직히 샘 멘데스 감독 좀 억울할듯기생충도 재밌게 봤지만, 이거 보고나서 좀 억울했겠다는 생각이 팍듬..', '동료를 잃고 목숨을 잃을 위기를 극복해나가며 달려 얻은 것은 전쟁의 무의미함과 허무함 뿐.', '전쟁이 끝나는 길은 마지막 한사람까지 죽는 것이다.', '기생충이 그냥 백인이 나온 헐리우드 영화였다면 절대 1917을 이기지 못했을것. 1917은 오히려 백인의 헐리우드 영화라 차별받음. 진짜 근래 보기 드믄 명작', '이 모든 것이 하루만의 일이라

In [3]:
movie_str_comment = ''
for comment in movie_comments:
    movie_str_comment += comment

print(movie_str_comment)

충무로: 이거 어케하는거냐?촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다.주인공을 카메라가 계속 따라가는데.. 세트장이 엄청 넓은거에 놀랐습니다. 직접 1차대전에 참전하는 느낌.. 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참.....닥터스트레인지의 판단력이 좋았다.솔직히 연출력만으로 최고의 영화다. 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다. 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽.아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음. 감독의 참신성이 빛을 발함. 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이...스포일러가 포함된 감상평입니다. 감상평 보기진짜 원테이크 장면 너무 긴거 아님?? 촬영하기 진짜 힘들었을 듯...솔직히 나는 기생충보다 이영화가 더 재미있었다.막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요!원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음스포일러가 포함된 감상평입니다. 감상평 보기일단..극장에서 보는걸  추천드립니다.카메라감독 상 줘야 할판  엄청나게  잘 찍음솔직히 샘 멘데스 감독 좀 억울할듯기생충도 재밌게 봤지만, 이거 보고나서 좀 억울했겠다는 생각이 팍듬..동료를 잃고 목숨을 잃을 위기를 극복해나가며 달려 얻은 것은 전쟁의 무의미함과 허무함 뿐.전쟁이 끝나는 길은 마지막 한사람까지 죽는 것이다.기생충이 그냥 백인이 나온 헐리우드 영화였다면 절대 1917을 이기지 못했을것. 1917은 오히려 백인의 헐리우드 영화라 차별받음. 진짜 근래 보기 드믄 명작이 모든 것이 하루만의 일이라니...스포일러가 포함된 감상평입니다. 감상평 보기전쟁씬 안나온다고 보지말라던 사람들 너무 한심하다병사를 따라가다보면 경험할 전쟁의 

In [4]:
r = '[-=+.#/\?:^$.@*\"~&%.,!\'\|\(\)\[\]\<\>]'

movie_str_comment = re.sub(r, '', movie_str_comment)
print(movie_str_comment)

충무로 이거 어케하는거냐촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다주인공을 카메라가 계속 따라가는데 세트장이 엄청 넓은거에 놀랐습니다 직접 1차대전에 참전하는 느낌 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참닥터스트레인지의 판단력이 좋았다솔직히 연출력만으로 최고의 영화다 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음 감독의 참신성이 빛을 발함 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이스포일러가 포함된 감상평입니다 감상평 보기진짜 원테이크 장면 너무 긴거 아님 촬영하기 진짜 힘들었을 듯솔직히 나는 기생충보다 이영화가 더 재미있었다막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음스포일러가 포함된 감상평입니다 감상평 보기일단극장에서 보는걸  추천드립니다카메라감독 상 줘야 할판  엄청나게  잘 찍음솔직히 샘 멘데스 감독 좀 억울할듯기생충도 재밌게 봤지만 이거 보고나서 좀 억울했겠다는 생각이 팍듬동료를 잃고 목숨을 잃을 위기를 극복해나가며 달려 얻은 것은 전쟁의 무의미함과 허무함 뿐전쟁이 끝나는 길은 마지막 한사람까지 죽는 것이다기생충이 그냥 백인이 나온 헐리우드 영화였다면 절대 1917을 이기지 못했을것 1917은 오히려 백인의 헐리우드 영화라 차별받음 진짜 근래 보기 드믄 명작이 모든 것이 하루만의 일이라니스포일러가 포함된 감상평입니다 감상평 보기전쟁씬 안나온다고 보지말라던 사람들 너무 한심하다병사를 따라가다보면 경험할 전쟁의 참상과 자연스레 따라오는 두려움과 먹먹함 이건 판타지를 제외한 영화의 기술이 구현

In [5]:
movie_new_comments = []
for comment in movie_comments:
    movie_new_comments.append(re.sub(r, '', comment))
    
print(movie_new_comments)

['충무로 이거 어케하는거냐', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다', '주인공을 카메라가 계속 따라가는데 세트장이 엄청 넓은거에 놀랐습니다 직접 1차대전에 참전하는 느낌 이때당시 군인들 20년뒤 히틀러라는 극악의 악마로 자식들 다 전장에 몰리고 더 최악의 고통을 받을껀데 참', '닥터스트레인지의 판단력이 좋았다', '솔직히 연출력만으로 최고의 영화다 마치 우리가 투명인간이 되서 그들을 지켜보는 듯했다 마지막장면은 영화 내내 쉴시간이 없었던것에 대한 피로감을 풀어줘서 기승전결도 완벽', '아카데미 작품상이야 작품전체를 보는거니 기생충이 받을수도 있다고 보는데 감독상은 1917 줬어도 할말 없음 감독의 참신성이 빛을 발함 결론적으로 아카데미 작품상과 감독상을 1917이 받았어도 아무 할말 없는 것이', '스포일러가 포함된 감상평입니다 감상평 보기', '진짜 원테이크 장면 너무 긴거 아님 촬영하기 진짜 힘들었을 듯', '솔직히 나는 기생충보다 이영화가 더 재미있었다', '막판에 횡으로 수많은 병사가 신호와 함께 우르르 튀어나올때 주인공 혼자 종으로 막 달리는 장면 와 진짜ㅋㅋㅋㅋ대박 무조건 아이맥스로 보세요', '원테이크 촬영신은 볼때마다 대단하다 후보에 오를만 했음', '스포일러가 포함된 감상평입니다 감상평 보기', '일단극장에서 보는걸  추천드립니다카메라감독 상 줘야 할판  엄청나게  잘 찍음', '솔직히 샘 멘데스 감독 좀 억울할듯기생충도 재밌게 봤지만 이거 보고나서 좀 억울했겠다는 생각이 팍듬', '동료를 잃고 목숨을 잃을 위기를 극복해나가며 달려 얻은 것은 전쟁의 무의미함과 허무함 뿐', '전쟁이 끝나는 길은 마지막 한사람까지 죽는 것이다', '기생충이 그냥 백인이 나온 헐리우드 영화였다면 절대 1917을 이기지 못했을것 1917은 오히려 백인의 헐리우드 영화라 차별받음 진짜 근래 보기 드믄 명작', '이 모든 것이 하루만의 일이라니', '스포일러가 포함된 감상평입니다 감상평 보기', '전쟁씬 안나온다고

# Text Preprocessing

## 형태소 분석

In [6]:
from nltk.tokenize import word_tokenize, sent_tokenize
from konlpy.tag import Okt

In [7]:
okt = Okt() 
# JPype 버전문제, 현재 JAVA_HOME 위치를 인식하지 못하는 문제
# 1. JAVA_HOME 설정 후 JPype 설치
# 2. JPype 버전과 python 버전 차이

## 형태소 추출

In [9]:
morphemes = []
for comment in movie_new_comments:
    for morphs in okt.morphs(comment):
        morphemes.append(morphs)

print(morphemes)

['충무로', '이', '거', '어케', '하는거냐', '촬영감독', '의', '영혼', '까지', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '같다', '주인공', '을', '카메라', '가', '계속', '따라가는데', '세트', '장이', '엄청', '넓은거에', '놀랐습니다', '직접', '1', '차', '대전', '에', '참전', '하는', '느낌', '이', '때', '당시', '군', '인들', '20년', '뒤', '히틀러', '라는', '극악', '의', '악마', '로', '자식', '들', '다', '전장', '에', '몰리', '고', '더', '최악', '의', '고통', '을', '받을껀데', '참', '닥터스', '트레인', '지', '의', '판단력', '이', '좋았다', '솔직히', '연출', '력', '만으로', '최고', '의', '영화', '다', '마치', '우리', '가', '투명인간', '이', '되서', '그', '들', '을', '지켜보는', '듯', '했다', '마지막', '장면', '은', '영화', '내내', '쉴', '시간', '이', '없었던것에', '대한', '피로', '감', '을', '풀어줘서', '기승', '전결', '도', '완벽', '아카데미', '작품', '상', '이야', '작품', '전체', '를', '보는거니', '기생충', '이', '받을수도', '있다고', '보는데', '감독', '상', '은', '1917', '줬어도', '할말', '없음', '감독', '의', '참신', '성', '이', '빛', '을', '발함', '결론', '적', '으로', '아카데미', '작품', '상과', '감독', '상', '을', '1917', '이', '받았어도', '아무', '할말', '없는', '것', '이', '스포일러', '가', '포함', '된', '감상', '평', '입니다', '감상', '평', '보기', '진짜', '

In [10]:
morphemes = []
for comment in movie_new_comments:
    for morphs in okt.nouns(comment):
        morphemes.append(morphs)

print(morphemes)

['충무로', '거', '어케', '촬영감독', '영혼', '마스터피스', '오스카', '작품', '주인공', '카메라', '계속', '세트', '직접', '차', '대전', '참전', '느낌', '때', '당시', '군', '히틀러', '극악', '악마', '자식', '전장', '몰리', '더', '최악', '고통', '받을껀데', '닥터스', '트레인', '의', '판단력', '연출', '최고', '영화', '마치', '우리', '투명인간', '그', '듯', '마지막', '장면', '영화', '내내', '시간', '대한', '피로', '감', '기승', '전결', '완벽', '아카데미', '작품', '작품', '전체', '기생충', '감독', '감독', '참신', '빛', '발함', '결론', '아카데미', '작품', '상과', '감독', '이', '아무', '것', '스포일러', '포함', '감상', '평', '감상', '평', '보기', '진짜', '테이크', '장면', '촬영', '진짜', '듯', '나', '기생충', '이영화', '더', '막판', '횡', '병사', '신호', '우르르', '때', '주인공', '혼자', '종', '막', '장면', '진짜', '대박', '무조건', '아이맥스', '테이크', '촬영', '볼때', '후보', '스포일러', '포함', '감상', '평', '감상', '평', '보기', '일단', '극장', '추천', '카메라감독', '상', '판', '샘', '데스', '감독', '좀', '기생충', '거', '좀', '생각', '팍듬', '동료', '목숨', '위기', '극복', '것', '전쟁', '뿐', '전쟁', '마지막', '사람', '것', '기생충', '그냥', '백인', '헐리우드', '영화', '절대', '이기', '은', '오히려', '백인', '헐리우드', '영화', '차별', '진짜', '근래', '보기', '드믄', '명작', '이', '모든', '것', '하루', '일이', '스포일러', '

## Stop words (불용어) 처리

In [11]:
stop_words = []
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append(line.rstrip('\n'))

In [13]:
results = []

for comment in movie_new_comments:
    for morphs in okt.morphs(comment):
        if morphs not in stop_words:
            results.append(morphs)
            
print(results)

['충무로', '이', '거', '어케', '하는거냐', '촬영감독', '영혼', '갈아', '넣은', '마스터피스', '오스카', '작품', '상', '탔어도', '할말', '없었을것', '주인공', '카메라', '계속', '따라가는데', '세트', '장이', '엄청', '넓은거에', '놀랐습니다', '직접', '1', '차', '대전', '참전', '하는', '느낌', '이', '당시', '군', '인들', '20년', '뒤', '히틀러', '라는', '극악', '악마', '자식', '다', '전장', '몰리', '고', '더', '최악', '고통', '받을껀데', '참', '닥터스', '트레인', '지', '판단력', '이', '좋았다', '솔직히', '연출', '력', '만으로', '최고', '영화', '다', '마치', '투명인간', '이', '되서', '지켜보는', '듯', '했다', '마지막', '장면', '은', '영화', '내내', '쉴', '이', '없었던것에', '대한', '피로', '감', '풀어줘서', '기승', '전결', '도', '완벽', '아카데미', '작품', '상', '이야', '작품', '전체', '보는거니', '기생충', '이', '받을수도', '있다고', '보는데', '감독', '상', '은', '1917', '줬어도', '할말', '없음', '감독', '참신', '성', '이', '빛', '발함', '결론', '적', '아카데미', '작품', '상과', '감독', '상', '1917', '이', '받았어도', '아무', '할말', '없는', '이', '스포일러', '포함', '된', '감상', '평', '입니다', '감상', '평', '보기', '진짜', '원', '테이크', '장면', '너무', '긴거', '아님', '촬영', '하기', '진짜', '힘들었을', '듯', '솔직히', '는', '기생충', '보다', '이영화', '더', '재미있었다', '막판', '횡', '수많은', '병사', '신호', '튀어나올', 

## Integer Encoding

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(results)

In [16]:
print(tokenizer.word_index)

{'이': 1, '영화': 2, '은': 3, '는': 4, '전쟁': 5, '한': 6, '테이크': 7, '촬영': 8, '도': 9, '연출': 10, '평': 11, '감상': 12, '진짜': 13, '전쟁영화': 14, '스토리': 15, '적': 16, '입니다': 17, '다': 18, '몰입': 19, '롱': 20, '정말': 21, '작품': 22, '기생충': 23, '최고': 24, '상': 25, '너무': 26, '기법': 27, '감': 28, '된': 29, '잘': 30, '느낌': 31, '만': 32, '내': 33, '수': 34, '주인공': 35, '보기': 36, '원': 37, '하는': 38, '이다': 39, '꼭': 40, '감동': 41, '포함': 42, '인': 43, '스포일러': 44, '게': 45, '영': 46, '안': 47, '1': 48, '하고': 49, '생각': 50, '장면': 51, '못': 52, '연기': 53, '말': 54, '있는': 55, '감독': 56, '1917': 57, '그냥': 58, '속': 59, '영화관': 60, '처음': 61, '중': 62, '할': 63, '긴장감': 64, '이런': 65, '본': 66, '더': 67, '점': 68, '사람': 69, '영상': 70, '보는': 71, '상미': 72, '없는': 73, '정도': 74, '보다': 75, '아이맥스': 76, '카메라': 77, '극장': 78, '아카데미': 79, '차': 80, '듯': 81, '보세요': 82, '보고': 83, '마지막': 84, '하나': 85, '끝': 86, '내내': 87, '같은': 88, '인생': 89, '고': 90, '배우': 91, '볼': 92, '명작': 93, '거': 94, '모든': 95, '해서': 96, '평점': 97, '걸': 98, '함': 99, '이건': 100, '중간': 101, '합니다': 102, '터'

In [17]:
print(tokenizer.word_counts)

OrderedDict([('충무로', 4), ('이', 1447), ('거', 65), ('어케', 11), ('하는거냐', 1), ('촬영감독', 17), ('영혼', 5), ('갈아', 5), ('넣은', 2), ('마스터피스', 13), ('오스카', 36), ('작품', 187), ('상', 174), ('탔어도', 4), ('할말', 11), ('없었을것', 1), ('주인공', 136), ('카메라', 79), ('계속', 45), ('따라가는데', 1), ('세트', 16), ('장이', 2), ('엄청', 27), ('넓은거에', 1), ('놀랐습니다', 2), ('직접', 19), ('1', 115), ('차', 76), ('대전', 34), ('참전', 10), ('하는', 130), ('느낌', 145), ('당시', 6), ('군', 6), ('인들', 1), ('20년', 2), ('뒤', 14), ('히틀러', 1), ('라는', 39), ('극악', 1), ('악마', 1), ('자식', 1), ('다', 206), ('전장', 47), ('몰리', 1), ('고', 70), ('더', 89), ('최악', 9), ('고통', 8), ('받을껀데', 1), ('참', 9), ('닥터스', 5), ('트레인', 5), ('지', 51), ('판단력', 1), ('좋았다', 16), ('솔직히', 35), ('연출', 278), ('력', 48), ('만으로', 8), ('최고', 177), ('영화', 1234), ('마치', 36), ('투명인간', 1), ('되서', 5), ('지켜보는', 3), ('듯', 76), ('했다', 55), ('마지막', 75), ('장면', 103), ('은', 432), ('내내', 74), ('쉴', 2), ('없었던것에', 1), ('대한', 16), ('피로', 4), ('감', 152), ('풀어줘서', 1), ('기승', 3), ('전결', 3), ('도', 281), ('완벽', 7), 

In [18]:
vocab_size = 50
tokenizer = Tokenizer()
tokenizer.fit_on_texts(results)

In [20]:
vocab_size = 50
words_frequency = [w for w, c in tokenizer.word_index.items() if c >= vocab_size + 1]
for w in words_frequency:
    del tokenizer.word_index[w]
    del tokenizer.word_counts[w]
    
print(tokenizer.word_index)
print()
print(tokenizer.word_counts)
print()
print(tokenizer.texts_to_sequences(results))

{'이': 1, '영화': 2, '은': 3, '는': 4, '전쟁': 5, '한': 6, '테이크': 7, '촬영': 8, '도': 9, '연출': 10, '평': 11, '감상': 12, '진짜': 13, '전쟁영화': 14, '스토리': 15, '적': 16, '입니다': 17, '다': 18, '몰입': 19, '롱': 20, '정말': 21, '작품': 22, '기생충': 23, '최고': 24, '상': 25, '너무': 26, '기법': 27, '감': 28, '된': 29, '잘': 30, '느낌': 31, '만': 32, '내': 33, '수': 34, '주인공': 35, '보기': 36, '원': 37, '하는': 38, '이다': 39, '꼭': 40, '감동': 41, '포함': 42, '인': 43, '스포일러': 44, '게': 45, '영': 46, '안': 47, '1': 48, '하고': 49, '생각': 50}

OrderedDict([('이', 1447), ('작품', 187), ('상', 174), ('주인공', 136), ('1', 115), ('하는', 130), ('느낌', 145), ('다', 206), ('연출', 278), ('최고', 177), ('영화', 1234), ('은', 432), ('감', 152), ('도', 281), ('기생충', 180), ('적', 215), ('스포일러', 118), ('포함', 120), ('된', 150), ('감상', 249), ('평', 254), ('입니다', 211), ('보기', 136), ('진짜', 234), ('원', 132), ('테이크', 322), ('너무', 169), ('촬영', 283), ('는', 417), ('잘', 147), ('생각', 107), ('전쟁', 415), ('한', 364), ('이다', 126), ('안', 117), ('수', 140), ('롱', 202), ('인', 119), ('기법', 153), ('내', 141),

In [21]:
vocab_size = 20
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token='OOV')
tokenizer.fit_on_texts(results)

print(tokenizer.texts_to_sequences(results))

[[1], [2], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [19], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [1], [1], [11], [1], [1], [1], [3], [19], [1], [1], [2], [1], [1], [1], [1], [1], [1], [4], [3], [1], [1], [2], [1], [1], [1], [1], [1], [1], [1], [10], [1], [1], [1], [1], [1], [1], [1], [1], [1], [2], [1], [1], [1], [1], [1], [4], [1], [1], [1], [1], [1], [1], [1], [2], [1], [1], [1], [17], [1], [1], [1], [1], [1], [1], [2], [1], [1], [1], [1], [2], [1], [1], [1], [13], [12], [18], [13], [12], [1], [14], [1], [8], [1], [1], [1], [1], [9], [1], [14], [1], [1], [1], [5], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [14], [1], [1], [1], [1], [1], [1], [8], [9], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [13], [12], [18], [13], [12], [1], [1], [1], [1], [1], [

## One-Hot Encoding

In [22]:
from tensorflow.keras.utils import to_categorical

In [24]:
encoded = tokenizer.texts_to_sequences([results])[0]
print(encoded)

[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 19, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 11, 1, 1, 1, 3, 19, 1, 1, 2, 1, 1, 1, 1, 1, 1, 4, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 17, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 13, 12, 18, 13, 12, 1, 14, 1, 8, 1, 1, 1, 1, 9, 1, 14, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 14, 1, 1, 1, 1, 1, 1, 8, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 13, 12, 18, 13, 12, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 6, 1, 1, 1, 6, 2, 1, 1, 1, 7, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 1, 1, 14, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 13, 12, 18, 13, 12, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 3, 1, 2, 1, 1, 1, 1, 1,

In [25]:
one_hot = to_categorical(encoded)
print(one_hot)
print(one_hot[0])

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# 단어 분리하기 (Byte Pair Encoding, BPE)

Integer Encoding : 단어별로 빈도수를 체크해서 순위를 매기는 Encoding

One-hot Encoding : 0 / 1을 이용해서 단어를 표현하는 Encoding, vector화해서 처리

OOV(Out of Vocabulary), UNK(Unknown Token) : Integer Encoding 결과에 포함되지 않은 단어일 때 발생

ex) Integer Encoding : 전체 100개 단어에 대해서 빈도수가 높은 30개 단어만 Integer Encoding 대상으로 사용

One-hot Encoding : vector 차원을 30으로 결정해서 빈도수가 높은 30개 단어가 아닌 다른 단어는 인식할 수 없는 문제가 발생

컴퓨터 입장에서 모르는 단어가 주어졌을 때 처리하는 방법으로 BPE 방법을 사용

BPE(Byte Pair Encoding) : 1994년에 제안된 데이터 압축 알고리즘

'aaabdaaabac'

- 1단계 : 'ZabdZabac', Z = aa
- 2단계 : 'ZYdZYac', Y = ab, Z = aa
- 3단계 : 'XdXac', X = ZY, Y = ab, Z = aa

# 자연어 처리에서의 BPE(Byte Pair Encoding)

자연어 처리에서의 BPE는 단어 분리(word segmentation) 알고리즘이다. 기존에 있던 단어를 분리한다는 의미이다.

BPE를 요약하면 글자(character) 단이에서 점차적으로 단어 집합(vocabulary)를 만들어내는 Bottom up 방식의 접근을 사용한다.

훈련 데이터에 있는 단어들을 모든 글자(character) 또는 유니코드(unicode) 단위로 단어 집합을 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합한다.

## 1) 기존 접근 방식

훈련 데이터에서 각 단어에 빈도수를 확인한 결과

low : 5<br>
lower : 2<br>
newest : 6<br>
widest : 3<br>

테스트 과정에서 lowest란 단어가 등장했을 때 -> OOV 문제가 발생

## 2) BPE 알고리즘 적용 방식

훈련 데이터에서 각 단어에 빈도수를 확인한 결과

low : 5<br>
lower : 2<br>
newest : 6<br>
widest : 3<br>

단어 집합을 참고로 한 초기 단어 집합(vocabulary)은 글자 단위로 분리된 상태이다.

단어 집합 (글자 단위)

l, o, w, e, r, n, w, s, t, i, d

BPE의 특징은 알고리즘의 동작은 몇 회 반복(iteration)할 것인지를 사용자가 정하는 점이다.

가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 몇 회 반복할 것인가를 정하는 것이다.

BPE 동작을 10회 수행한다고 가정

1회 동작 : 단어 집합을 참고로 하여 빈도수가 9로 가장 높은 (e, s)의 쌍을 es로 통합

단어 집합 (단어 단위)

l o w : 5<br>
l o w e r : 2<br>
n e w es t : 6<br>
wid es t : 3<br>

단어 집합(글자 단위)

l, o, w, e, r, n, w, s, t, i, d, es

2회 동작 : 단어 집합을 참고하여 빈도수가 9로 가장 높은 (es, t)의 쌍을 est로 통합

단어 집합 (단어 단위)

l o w : 5<br>
l o w e r : 2<br>
n e w est : 6<br>
wid est : 3<br>

단어 집합(글자 단위)

l, o, w, e, r, n, w, s, t, i, d, es, est<br>

3회 동작 : 단어 집합을 참고하여 빈도수가 7로 가장 높은 (l, o)의 쌍을 lo로 통합

단어 집합 (단어 단위)

lo w : 5<br>
lo w e r : 2<br>
n e w est : 6<br>
wid est : 3<br>

단어 집합(글자 단위)

l, o, w, e, r, n, w, s, t, i, d, es, est, lo<br>

10회 동작 후 

단어 집합 (단어 단위)

low : 5<br>
low e r : 2<br>
newest : 6<br>
widest : 3<br>

단어 집합(글자 단위)

l, o, w, e, r, n, w, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest<br>

테스트 과정에서 'lowest'란 단어가 등장한다면, 기존 Integer Encoding에서는 OOV로 처리

BPE 적용시에는 OOV로 처리하지 않는다.



## BPE 코드 실습

In [26]:
import re, collections

BPE를 수행할 횟수 지정

In [27]:
num_merges = 10

In [28]:
vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>' : 6,
         'w i d e s t </w>' : 3
        }

BPE 적용한 코드는 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 num_merges회 반복한다.

In [29]:
# 글자별 빈도수 계산
def get_stats(vocab):
    pairs = collections.defaultdict(int) # key 없이도 사용할 수 있는 딕셔너리
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i+1]] += freq
    
    return pairs

In [30]:
# 글자 빈도수에 따른 유니그램 작성
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile('(?<!\S)' + bigram + '(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out

In [31]:
for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print(best)

('e', 's')
('es', 't')
('est', '</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')


In [32]:
print(vocab)

{'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


## WPM(Wordpiece Model)

WPM 아이디어를 제시한 논문
https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf

WPM 아이디어를 제시한 논문을 토대로 구글에서 변형하여 번역기에 사용한 Model
https://arxiv.org/pdf/1609.08144.pdf

WPM은 BPE의 변형된 형태

WPM을 수행하기 이전 문장 : Jet makers feud over width with big orders at stake

WPM을 수행한 경우 : _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake

## Google의 Sentencepiece

논문 : https://arxiv.org/pdf/1808.06226.pdf
Sentencepiece github : https://github.com/google/sentencepiece

Sentencepiece 이점 : 단어 분리 알고리즘을 사용하기 위해 데이터에 단어 토큰화를 먼저 진행한 상태여야 하므로 모든 언어에서 사용하는데는 어려움이 있으나, sentencepiece 사전 토큰화 작업없이 단어 분리 토큰화를 수행하는 알고리즘으로 언어에 종속되지 않는 알고리즘이다.

## 데이터 분리를 위한 Python code 실습

### 1) zip 함수 이용하여 분리하기

In [33]:
X, y = zip(['a', 1], ['b', 2], ['c', 3])
print(X)
print(y)

('a', 'b', 'c')
(1, 2, 3)


In [34]:
sequences = [ ['a', 1], ['b', 2], ['c', 3]]
X, y = zip(*sequences)
print(X)
print(y)

('a', 'b', 'c')
(1, 2, 3)


### 2) DataFrame을 이용한 분리

In [35]:
import pandas as pd

In [38]:
values = [ ['당신에게 드리는 마지막 혜택!', 1],
          ['내일 뵐 수 있을지 확인 부탁드...', 0],
          ['길동씨, 잘 지내시죠? 오랜만입...', 0],
          ['(광고) AI로 주가를 예측할 수 있다!', 1]
        ]
columns = ['메일 본문', '스팸 메일 유무']

In [39]:
df = pd.DataFrame(values, columns = columns)
df

,메일 본문,스팸 메일 유무
0,당신에게 드리는 마지막 혜택!,1
1,내일 뵐 수 있을지 확인 부탁드...,0
2,"길동씨, 잘 지내시죠? 오랜만입...",0
3,(광고) AI로 주가를 예측할 수 있다!,1


In [40]:
X = df['메일 본문']
y = df['스팸 메일 유무']

In [41]:
print(X)

0          당신에게 드리는 마지막 혜택!
1      내일 뵐 수 있을지 확인 부탁드...
2      길동씨, 잘 지내시죠? 오랜만입...
3    (광고) AI로 주가를 예측할 수 있다!
Name: 메일 본문, dtype: object


In [42]:
print(y)

0    1
1    0
2    0
3    1
Name: 스팸 메일 유무, dtype: int64


### 3) numpy를 이용하여 분리

In [43]:
import numpy as np

In [45]:
ar = np.arange(0, 16).reshape((4, 4))
print(ar)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


In [46]:
X = ar[:, :3]
print(X)

[[ 0  1  2]
 [ 4  5  6]
 [ 8  9 10]
 [12 13 14]]


In [48]:
y = ar[:, 3]
print(y)

[ 3  7 11 15]


## 테스트 데이터 분리하기

### scikit-learn 이용한 분리

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1234)

In [53]:
X, y = np.arange(10).reshape((5, 2)), range(5)
print(X)
print(list(y))

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
[0, 1, 2, 3, 4]


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

In [55]:
print(X_train)
print(X_test)

[[2 3]
 [4 5]
 [6 7]]
[[8 9]
 [0 1]]
